### spaCy

Results not too great with TfidfVectorizer alone or in tandem with Gensim simple_preprocess.
Ideas for improving are more specialized NLP and/or gathering better book synopsis data.

But first... gonna see how spaCy does for me

In [7]:
import numpy as np
import pandas as pd
import re
import spacy
import enchant

In [12]:
df = pd.read_csv('master_books.csv')
print(df.shape)
df.head()

(19707, 6)


,title,author,rating,description,language,isbn
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"4.33 avg rating — 6,191,931 ratings",WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,0439023483 (ISBN13: 9780439023481)
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,"4.50 avg rating — 2,414,060 ratings",There is a door at the end of a silent corrido...,English,0439358078 (ISBN13: 9780439358071)
2,To Kill a Mockingbird,Harper Lee,"4.28 avg rating — 4,348,656 ratings",The unforgettable novel of a childhood in a sl...,English,NaN
3,Pride and Prejudice,Jane Austen,"4.26 avg rating — 2,886,703 ratings",Alternate cover edition of ISBN 9780679783268S...,English,NaN
4,"Twilight (Twilight, #1)",Stephenie Meyer,"3.59 avg rating — 4,815,701 ratings",About three things I was absolutely positive.\...,English,0316015849 (ISBN13: 9780316015844)


In [13]:
df.isnull().sum()

title             0
author            0
rating            0
description     676
language       1617
isbn           4596
dtype: int64

In [15]:
mask = df.description.isnull()

df[~mask].shape

(19031, 6)

In [16]:
df = df.dropna(subset=['description'])
df.shape

(19031, 6)

In [17]:
df['text'] = [re.sub('\W+', ' ', text.lower()) for text in df['description']]

C:\Users\jayja\anaconda3\envs\books-kernel\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
# ~16000 books with 'English' as the language - save those
df['language'].value_counts()

English                                15812
Arabic                                   408
Spanish                                  261
French                                   207
German                                   156
Indonesian                               154
Portuguese                               110
Italian                                   86
Dutch                                     86
Turkish                                   73
Persian                                   46
Polish                                    40
Czech                                     33
Romanian                                  29
Bulgarian                                 28
Malay                                     25
Swedish                                   24
Japanese                                  22
Greek, Modern (1453-)                     20
Russian                                   16
Urdu                                      16
Estonian                                  12
Vietnamese

In [19]:
df = df[df['language'] == 'English']
df.shape

(15812, 7)

In [6]:
english = enchant.Dict("en_US")

def check_english(book, english=english):
    ten_words = book.split()[:10]
    
    word_is_english = []
    
    for word in ten_words:
        word_is_english.append(english.check(word))
    
    english_words = word_is_english.count(True)
    return english_words / len(word_is_english)

In [20]:
df['is_english'] = df['text'].apply(check_english)

df.head(10)

,title,author,rating,description,language,isbn,text,is_english
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"4.33 avg rating — 6,191,931 ratings",WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,0439023483 (ISBN13: 9780439023481),winning means fame and fortune losing means ce...,1.0
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,"4.50 avg rating — 2,414,060 ratings",There is a door at the end of a silent corrido...,English,0439358078 (ISBN13: 9780439358071),there is a door at the end of a silent corrido...,1.0
2,To Kill a Mockingbird,Harper Lee,"4.28 avg rating — 4,348,656 ratings",The unforgettable novel of a childhood in a sl...,English,NaN,the unforgettable novel of a childhood in a sl...,1.0
3,Pride and Prejudice,Jane Austen,"4.26 avg rating — 2,886,703 ratings",Alternate cover edition of ISBN 9780679783268S...,English,NaN,alternate cover edition of isbn 9780679783268s...,0.8
4,"Twilight (Twilight, #1)",Stephenie Meyer,"3.59 avg rating — 4,815,701 ratings",About three things I was absolutely positive.\...,English,0316015849 (ISBN13: 9780316015844),about three things i was absolutely positive f...,0.9
5,The Book Thief,Markus Zusak (Goodreads Author),"4.37 avg rating — 1,767,200 ratings",It is 1939. Nazi Germany. The country is holdi...,English,0375831002 (ISBN13: 9780375831003),it is 1939 nazi germany the country is holding...,0.8
6,The Chronicles of Narnia (Chronicles of Narnia...,C.S. Lewis,"4.26 avg rating — 498,771 ratings","Journeys to the end of the world, fantastic cr...",English,NaN,journeys to the end of the world fantastic cre...,1.0
7,Animal Farm,George Orwell,"3.94 avg rating — 2,631,549 ratings",Librarian's note: There is an Alternate Cover ...,English,0451526341 (ISBN13: 9780451526342),librarian s note there is an alternate cover e...,1.0
8,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,J.R.R. Tolkien,"4.60 avg rating — 107,831 ratings","This four-volume, boxed set contains J.R.R. To...",English,0345538374 (ISBN13: 9780345538376),this four volume boxed set contains j r r tolk...,0.9
9,Gone with the Wind,Margaret Mitchell,"4.30 avg rating — 1,054,418 ratings","Scarlett O'Hara, the beautiful, spoiled daught...",English,0446675539 (ISBN13: 9780446675536),scarlett o hara the beautiful spoiled daughter...,0.8


In [21]:
mask = df['is_english'] >= .5

df = df[mask]

In [22]:
df.shape

(15742, 8)

In [23]:
def get_description_length(book):
    return len(book.split())

df['descrip_len'] = df['text'].apply(get_description_length)

In [24]:
len(df.query("descrip_len <= 6"))

33

In [26]:
# dropping books with descriptions shorter than 7 words
mask = df['descrip_len'] > 6

df = df[mask]

In [27]:
df.shape

(15709, 9)

In [28]:
seen = set()
def find_doubles(title, author, seen=seen):
    title_author = (title, author)
    if title_author not in seen:
        seen.add(title_author)
        return False
    else:
        return True

In [29]:
df['double'] = df.apply(lambda x: find_doubles(x.title, x.author), axis=1)

In [30]:
df['double'].value_counts()

False    12240
True      3469
Name: double, dtype: int64

In [31]:
mask = ~df['double']
df = df[mask]

In [32]:
df.shape

(12240, 10)